In [1]:


import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# ==============================================================================
# PASSO 2: CARREGAMENTO E TRANSFORMAÇÃO DOS DADOS
# ==============================================================================
file_path = 'comprovantes_pix_10000_anomalias.csv'
try:
    df_real = pd.read_csv(file_path, delimiter=';')
    print(f"Arquivo '{file_path}' carregado com sucesso. {len(df_real)} linhas encontradas.")

    df_real['DataHora'] = pd.to_datetime(df_real['DataHora'], errors='coerce')
    linhas_originais = len(df_real)
    df_real.dropna(subset=['DataHora'], inplace=True)
    if len(df_real) < linhas_originais:
        print(f"{linhas_originais - len(df_real)} linhas com datas inválidas foram removidas.")

    df_transformed = pd.DataFrame()

    bins_amount = [0, 50, 1000, float('inf')]
    labels_amount = ['valor_baixo', 'valor_medio', 'valor_alto']
    df_transformed['valor_categoria'] = pd.cut(df_real['Valor'], bins=bins_amount, labels=labels_amount, right=False)

    df_real['hora_transacao'] = df_real['DataHora'].dt.hour
    bins_hour = [0, 6, 12, 18, 24]
    labels_hour = ['madrugada', 'manha', 'tarde', 'noite']
    df_transformed['hora_categoria'] = pd.cut(df_real['hora_transacao'], bins=bins_hour, labels=labels_hour, right=False)

    df_transformed['status_anomalia'] = df_real['Anomalia'].apply(lambda x: 'fraude' if x else 'nao_fraude')

    # ==============================================================================
    # PASSO 3: MINERAÇÃO DAS REGRAS (COM PARÂMETROS DE ALTA SENSIBILIDADE)
    # ==============================================================================
    transactions_list = df_transformed.apply(lambda row: row.dropna().tolist(), axis=1).tolist()

    te = TransactionEncoder()
    te_ary = te.fit(transactions_list).transform(transactions_list)
    df_encoded = pd.DataFrame(te_ary, columns=te.columns_)

    # Parâmetros de alta sensibilidade para encontrar padrões raros
    frequent_itemsets = apriori(df_encoded, min_support=0.001, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.01)
    
    # ==============================================================================
    # PASSO 4: FILTRAGEM PRECISA E EXPORTAÇÃO DO RESULTADO FINAL
    # ==============================================================================
    regras_de_fraude_apenas = rules[rules['antecedents'].apply(lambda x: 'fraude' in x) | rules['consequents'].apply(lambda x: 'fraude' in x)]
    regras_de_fraude_sorted = regras_de_fraude_apenas.sort_values(by='lift', ascending=False)

    print("\n==========================================================")
    print("      RESULTADO FINAL: REGRAS DE ASSOCIAÇÃO DE FRAUDE")
    print("==========================================================")
    if regras_de_fraude_sorted.empty:
        print("Nenhuma regra de fraude foi encontrada.")
    else:
        print(regras_de_fraude_sorted[['antecedents', 'consequents', 'support', 'confidence', 'lift']])

        output_filename = 'regras_de_fraude_encontradas.csv'
        regras_de_fraude_sorted.to_csv(output_filename, index=False, sep=';', encoding='utf-8-sig')
        print(f"\nResultados salvos com sucesso no arquivo: '{output_filename}'")

except FileNotFoundError:
    print(f"ERRO: O arquivo '{file_path}' não foi encontrado.")
except Exception as e:
    print(f"Ocorreu um erro inesperado: {e}")

Arquivo 'comprovantes_pix_10000_anomalias.csv' carregado com sucesso. 10000 linhas encontradas.
24 linhas com datas inválidas foram removidas.

      RESULTADO FINAL: REGRAS DE ASSOCIAÇÃO DE FRAUDE
              antecedents              consequents   support  confidence  \
51   (manha, valor_baixo)                 (fraude)  0.001002    0.384615   
53               (fraude)     (manha, valor_baixo)  0.001002    0.131579   
54          (valor_baixo)          (manha, fraude)  0.001002    0.094340   
50        (manha, fraude)            (valor_baixo)  0.001002    0.416667   
5                (fraude)            (valor_baixo)  0.002205    0.289474   
6           (valor_baixo)                 (fraude)  0.002205    0.207547   
52  (fraude, valor_baixo)                  (manha)  0.001002    0.454545   
1                (fraude)                  (manha)  0.002406    0.315789   
44   (valor_alto, fraude)              (madrugada)  0.001203    0.285714   
0                (fraude)              (ma